# VeloVAE (fullvb) benchmark on dyngen data

Notebook benchmarks velocity and latent time inference using VeloVAE (fullvb) on dyngen-generated data.

## Library imports

In [1]:
import velovae as vv

import numpy as np
import pandas as pd
import torch

import anndata as ad
import scvelo as scv

from rgv_tools import DATA_DIR
from rgv_tools.benchmarking import get_velocity_correlation

/home/icb/yifan.chen/miniconda3/envs/regvelo-py310-velovae/lib/python3.10/site-packages/velovae/model/model_util.py:12: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import trange


## General settings

In [2]:
scv.settings.verbosity = 3

In [3]:
COMPLEXITY = "complexity_1"

## Constants

In [4]:
torch.manual_seed(0)
np.random.seed(0)

In [5]:
DATASET = "dyngen"

In [6]:
SAVE_DATA = True
if SAVE_DATA:
    (DATA_DIR / DATASET / COMPLEXITY / "results").mkdir(parents=True, exist_ok=True)
    (DATA_DIR / DATASET / COMPLEXITY / "processed" / "velovae_fullvb_vae").mkdir(parents=True, exist_ok=True)

In [7]:
SAVE_DATASETS = True
if SAVE_DATASETS:
    (DATA_DIR / DATASET / COMPLEXITY / "trained_velovae_fullvb").mkdir(parents=True, exist_ok=True)

## Velocity pipeline

In [8]:
velocity_correlation = []

cnt = 0
for filename in (DATA_DIR / DATASET / COMPLEXITY / "processed").iterdir():
    if filename.suffix != ".zarr":
        continue

    print(f"Run {cnt}, file {filename}.")
    adata = ad.io.read_zarr(filename)

    try:
        rate_prior = {"alpha": (0.0, 1.0), "beta": (0.0, 0.5), "gamma": (0.0, 0.5)}
        vae = vv.VAE(adata, tmax=20, dim_z=5, device="cuda:0", full_vb=True, rate_prior=rate_prior)
        config = {}
        vae.train(adata, config=config, plot=False, embed="pca")

        simulation_id = int(filename.stem.removeprefix("simulation_"))
        # Output velocity to adata object
        vae.save_anndata(adata, "fullvb", DATA_DIR / DATASET / COMPLEXITY / "processed" / "velovae_fullvb_vae", file_name=f"velovae_fullvb_{simulation_id}.h5ad")

        adata.layers["velocity"] = adata.layers["fullvb_velocity"].copy()

        # save data
        adata.write_zarr(DATA_DIR / DATASET / COMPLEXITY / "trained_velovae_fullvb" / f"trained_{simulation_id}.zarr")

        velocity_correlation.append(
            get_velocity_correlation(
                ground_truth=adata.layers["true_velocity"], estimated=adata.layers["velocity"], aggregation=np.mean
            )
        )

    except Exception as e:  # noqa: BLE001
        # Append np.nan in case of an error and optionally log the error
        print(f"An error occurred: {e}")
        velocity_correlation.append(np.nan)

    cnt += 1

Run 0, file /ictstr01/groups/ml01/workspace/yifan.chen/regvelo_reproducibility/data/dyngen/complexity_1/processed/simulation_29.zarr.
Estimating ODE parameters...


  0%|          | 0/362 [00:00<?, ?it/s]

Detected 347 velocity genes.
Estimating the variance...


  0%|          | 0/362 [00:00<?, ?it/s]

Initialization using the steady-state and dynamical models.
Reinitialize the regular ODE parameters based on estimated global latent time.


...

Initialization using the steady-state and dynamical models.
Reinitialize the regular ODE parameters based on estimated global latent time.


  0%|          | 0/494 [00:00<?, ?it/s]

3 clusters detected based on gene co-expression.
KS-test result: [1. 1. 1.]
Assign cluster 1 to repressive
Initial induction: 324, repression: 170/494
Learning Rate based on Data Sparsity: 0.0003
--------------------------- Train a VeloVAE ---------------------------
*********        Creating Training/Validation Datasets        *********
*********                      Finished.                      *********
*********                 Creating optimizers                 *********
*********                      Finished.                      *********
*********                    Start training                   *********
*********                      Stage  1                       *********
Total Number of Iterations Per Epoch: 6, test iteration: 10
*********       Stage 1: Early Stop Triggered at epoch 824.       *********
*********                      Stage  2                       *********
*********             Velocity Refinement Round 1             *********


  0%|          | 0/1000 [00:00<?, ?it/s]

Percentage of Invalid Sets: 0.118
Average Set Size: 19
*********     Round 1: Early Stop Triggered at epoch 1041.    *********
Change in noise variance: 0.0273
*********             Velocity Refinement Round 2             *********
*********     Round 2: Early Stop Triggered at epoch 1113.    *********
Change in noise variance: 0.0124
Change in x0: 0.2851
*********             Velocity Refinement Round 3             *********
*********     Round 3: Early Stop Triggered at epoch 1132.    *********
Change in noise variance: 0.0018
Change in x0: 0.1825
*********             Velocity Refinement Round 4             *********
*********     Round 4: Early Stop Triggered at epoch 1156.    *********
Change in noise variance: 0.0004
Change in x0: 0.1316
*********             Velocity Refinement Round 5             *********
*********     Round 5: Early Stop Triggered at epoch 1182.    *********
Change in noise variance: 0.0000
Change in x0: 0.1173
*********             Velocity Refinement Round 

## Data saving

In [10]:
if SAVE_DATA:
    pd.DataFrame({"velocity": velocity_correlation}).to_parquet(
        path=DATA_DIR / DATASET / COMPLEXITY / "results" / "velovae_fullvb_correlation.parquet"
    )